Na początku zamontowaliśmy dysk Google.

In [ ]:
# import google drive
from google.colab import drive

# mount google drive
drive.mount('/content/drive')


Następnie rozpakowaliśmy folder z danymi do `/content/data_dir`

In [ ]:
# unzip data.zip
!unzip "/content/drive/MyDrive/studies/ai-project-3/data.zip" -d /content/data_dir


Dalszym etapem było zaimportowanie potrzebnych bibliotek.

In [ ]:
# import
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import matplotlib.pyplot as plt
import random
import cv2
from sklearn.model_selection import train_test_split


Zapisaliśmy do zmiennej informację o ścieżce z danymi.

In [ ]:
# data_dir
data_dir = "/content/data_dir"
data_dir = pathlib.Path(data_dir)


Obliczyliśmy ilość wsyztskich zdjęć.

In [ ]:
# image_count
image_count = len(list(data_dir.glob('*/*.*')))


Ustawiliśmy `batch_size`, `img_height` i `img_width`.

In [ ]:
# configs
batch_size = 3
img_height = 128
img_width = 128


Utworzyliśmy training dataset z 80% zbioru danych.

In [ ]:
# train dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
  

Utworzyliśmy validation dataset z 20% zbioru danych.

In [ ]:
# validation dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Zapisaliśmy do tablicy listę kategorii na podstawie train dataset.

In [ ]:
# class names
class_names = train_ds.class_names


Jako że zdjęcia w AI/ML nie mogą być obsługiwane w skali [0, 255] na kolor, tylko w skali [0, 1], to zmieniamy je.

In [ ]:
# serialize data
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))


Optymalizacja dataset'ów.

In [ ]:
# performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Zebraliśmy wszystkie ścieżki zdjęć z każdej kategorii.

In [ ]:
# imagePaths
imagePaths = []

for k, class_name in enumerate(class_names):
    for f in os.listdir(str(data_dir)+"/"+class_name):
        imagePaths.append([str(data_dir)+"/"+class_name+'/'+f, k])

random.shuffle(imagePaths)


Zmieniliśmy rozmiary zdjęć, tak żeby każde miało taki sam oraz zapisaliśmy do jednej tablicy zdjęcia, a do drugiej odpowiadające im labele.

In [ ]:
# data and labels
data = []
labels = []

for imagePath in imagePaths:
    image = cv2.imread(imagePath[0])
    image = cv2.resize(image, (img_width, img_height))
    data.append(image)
    
    label = imagePath[1]
    labels.append(label)


Jako że zdjęcia w AI/ML nie mogą być obsługiwane w skali [0, 255] na kolor, tylko w skali [0, 1], to zmieniamy je.

In [ ]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)


Przygotowujemy dane treningowe i testowe oraz odpowiadujące im labele. (różnica między poprzednimi wersjami, a tą jest taka, że wykorzytujemy tutaj tylko dane testowe po wytrenowaniu modelu)

In [ ]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)


Tworzymy model `U-Net`.

In [ ]:
# create model
model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),

  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(1024, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(1024, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(512, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(256, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid'),

  # tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(len(class_names))
])


Kompilujemy model przy użyciu optymalizera `adam` i metryki `accuracy`.

In [ ]:
# compile model
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


Trenujemy model przy użyciu train dataset i validation dataset. Trening wykonujemy przez 3 epoki.

In [ ]:
# train model
epochs = 3
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


Sprawdzamy precyzję modelu na podstawie testowych danych.

In [ ]:
# Evaluate accuracy
test_loss, test_acc = model.evaluate(testX, testY, verbose=2)
print('\nTest accuracy:', test_acc)


Na danych testowych predyktujemy model, dzięki czemu otrzymujemy otrzymujemy informacje, które obrazki pasowały do jakich kategorii i z jakim powodzeniem.

In [ ]:
# Make predictions
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(testX)



Tworzymy pomocnicze funkcje.

In [ ]:
# helper
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(len(class_names)))
  plt.yticks([])
  thisplot = plt.bar(range(len(class_names)), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Tworzymy wykres z kilkoma przykładowymi obrazkami i kwalifikacją przyznaną im wraz z wartością procentową ukazującą jak dobrze został obrazek rozpoznany w danej kategorii.

In [ ]:
# Verify predictions
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], testY, testX)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], testY)
plt.tight_layout()
plt.show()
